# 使用pytest-django进行测试

## 相关教程

- [How to Provide Test Fixtures for Django Models in Pytest](https://realpython.com/django-pytest-fixtures/)
- [Testing Your Django App With Pytest](https://djangostars.com/blog/django-pytest-testing/)

## 安装

最简单的方法，运行`pip install pytest-django`以后，在项目的根目录下新建一个pytest.ini文件，配置如下：
```ini
[pytest]
DJANGO_SETTINGS_MODULE = yourproject.settings
```
然后在根目录下直接运行pytest即可，pytest会搜集各个目录下的测试用例。

### 数据库辅助

1. 可以使用`@pytest.mark.django_db`标记，这样就可以在测试用例里面链接数据库了。
2. 如果是在自定义的fixture里面连接数据库，则使用`db`和`transactional_db`或者`django_db_reset_sequences` fixture。一般情况下，使用`db`就够了。

### 客户端辅助

如果要模拟客户端发起的请求，可以使用`client` fixture：
```python
import pytest

from django.urls import reverse

@pytest.mark.django_db
def test_view(client):
   url = reverse('homepage-url')
   response = client.get(url)
   assert response.status_code == 200
```
有时候只想测试视图的逻辑，可是普通用户还需要先登录授权，此时可以使用`admin_client` fixture：
```python
import pytest

from django.urls import reverse


@pytest.mark.django_db
def test_unauthorized(client):
   url = reverse('superuser-url')
   response = client.get(url)
   assert response.status_code == 401


@pytest.mark.django_db
def test_superuser_view(admin_client):
   url = reverse('superuser-url')
   response = admin_client.get(url)
   assert response.status_code == 200
```

### 创建用户

有2个fixture和用户相关，一个是`django_user_model`，通过配置文件获取user的模型，这样就不用再手动的导入用户模型了。一个是`admin_user`，直接返回管理员user。

也可以使用工厂函数自定义fixture来创建user，比如：
```python
from typing import List, Optional

import pytest
from django.contrib.auth.models import User, Group, Permission

@pytest.fixture
def app_user_group(db) -> Group:
    group = Group.objects.create(name="app_user")
    change_user_permissions = Permission.objects.filter(
        codename__in=["change_user", "view_user"],
    )
    group.permissions.add(*change_user_permissions)
    return group

@pytest.fixture
def app_user_factory(db, app_user_group: Group):
    # 使用闭包来传递参数，因为普通的fixture不能接受自己的参数
    def create_app_user(
        username: str,
        password: Optional[str] = None,
        first_name: Optional[str] = "first name",
        last_name: Optional[str] = "last name",
        email: Optional[str] = "foo@bar.com",
        is_staff: str = False,
        is_superuser: str = False,
        is_active: str = True,
        groups: List[Group] = [],
    ) -> User:
        user = User.objects.create_user(
            username=username,
            password=password,
            first_name=first_name,
            last_name=last_name,
            email=email,
            is_staff=is_staff,
            is_superuser=is_superuser,
            is_active=is_active,
        )
        user.groups.add(app_user_group)
        # Add additional groups, if provided.
        user.groups.add(*groups)
        return user
    return create_app_user

@pytest.fixture
def user_A(db, app_user_factory) -> User:
    return app_user_factory("A")

@pytest.fixture
def user_B(db, app_user_factory) -> User:
    return app_user_factory("B")

def test_should_create_user_in_app_user_group(
    user_A: User,
    app_user_group: Group,
) -> None:
    assert user_A.groups.filter(pk=app_user_group.pk).exists()

def test_should_create_two_users(user_A: User, user_B: User) -> None:
    assert user_A.pk != user_B.pk
```

## 使用pytest测试Django REST Framework

### rest framework的api

rest框架提供一个client，可以自定义一个fixture方便使用：
```python
import pytest


@pytest.fixture
def api_client():
   from rest_framework.test import APIClient
   return APIClient()
```

### rest framework登录

rest framework一般使用token登录，测试方法如下：
```python
import pytest
from django.urls import reverse
from rest_framework.authtoken.models import Token


@pytest.fixture
def get_or_create_token(db, create_user):
   user = create_user()
   token, _ = Token.objects.get_or_create(user=user)
   return token


@pytest.mark.django_db
def test_unauthorized_request(api_client, get_or_create_token):
   url = reverse('need-token-url')
   token = get_or_create_token()
   api_client.credentials(HTTP_AUTHORIZATION='Token ' + token.key)
   response = api_client.get(url)
   assert response.status_code == 200
```

可以继续改造成fixture，避免每个测试用例都要重新定义一次：
```python
@pytest.fixture
def api_client_with_credentials(
   db, create_user, api_client
):
   user = create_user()
   api_client.force_authenticate(user=user)
   yield api_client
   api_client.force_authenticate(user=None)
```

### 参数化测试

```python
import pytest


@pytest.mark.django_db
@pytest.mark.parametrize(
   'email, password, status_code', [
       (None, None, 400),
       (None, 'strong_pass', 400),
       ('user@example.com', None, 400),
       ('user@example.com', 'invalid_pass', 400),
       ('user@example.com, 'strong_pass', 201),
   ]
)
def test_login_data_validation(
   email, password, status_code, api_client
):
   url = reverse('login-url')
   data = {
       'email': email,
       'password': password
   }
   response = api_client.post(url, data=data)
   assert response.status_code == status_code
```

### 使用mock

mock有好几种方式，比如官方的`unittest.mock`，pytest的`monkeypatch`和第三方包`pytest-mock`。